# Data Generation

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

orders_daily_data = [
    ("o1", "c1", "SHIPPED", 120.0, "2025-02-01 10:00:00", "2025-02-01 11:00:00"),
    ("o2", "c2", "DELIVERED", 200.0, "2025-02-01 12:00:00", "2025-02-01 12:30:00"),
    ("o3", "c3", "PENDING", -50.0, "2025-02-01 13:00:00", "2025-02-01 13:05:00"),  # invalid negative
    (None, "c4", "SHIPPED", 80.0, "2025-02-01 14:00:00", "2025-02-01 14:10:00"), # null order_id
    ("o1", "c1", "DELIVERED", 120.0, "2025-02-01 10:00:00", "2025-02-01 15:00:00"), # newer update
    ("o4", "c5", "PENDING", 300.0, "2025-01-29 09:00:00", "2025-02-01 16:00:00"), # late arriving
    ("o5", "c6", "CANCELLED", 150.0, "2025-02-01 17:00:00", "2025-02-01 17:05:00")
]

orders_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_status", StringType(), True),
    StructField("order_amount", DoubleType(), True),
    StructField("order_timestamp", StringType(), True),
    StructField("last_updated", StringType(), True),
])

orders_daily_df = spark.createDataFrame(orders_daily_data, schema=orders_schema)

orders_daily_df.write.mode("overwrite").option("header", True).csv("/Volumes/pyspark/bronze/raw_ingestion/csv/orders_daily_csv")

In [0]:
from pyspark.sql.functions import to_timestamp

orders_existing_data = [
    ("o1", "c1", "PENDING", 120.0, "2025-02-01 10:00:00", "2025-02-01 10:30:00"),
    ("o2", "c2", "SHIPPED", 200.0, "2025-02-01 12:00:00", "2025-02-01 12:15:00"),
    ("o6", "c7", "DELIVERED", 400.0, "2025-01-30 08:00:00", "2025-01-30 09:00:00")
]

orders_existing_df = spark.createDataFrame(orders_existing_data, schema=orders_schema)

orders_existing_df = (
    orders_existing_df
        .withColumn("order_timestamp", to_timestamp(col("order_timestamp")))
        .withColumn("last_updated", to_timestamp(col("last_updated")))
)

orders_existing_df.write.format("delta").mode("overwrite").saveAsTable("pyspark.silver.t2_orders")


# Pair Programming Task 2 — Incremental Pipeline + Delta

## 🎯 Scenario

You are building a pipeline for an **Orders system**.

You receive daily data from an API.

Each file contains:

### 📁 `orders_daily.csv`

```
order_id (string)
customer_id (string)
order_status (string)
order_amount (double)
order_timestamp (string)
last_updated (string)
```

### ⚠️ Important

* Multiple updates for same order_id may arrive.
* Late-arriving updates are possible.
* Some records may already exist in target.
* Some orders may change status from:
  PENDING → SHIPPED → DELIVERED → CANCELLED

---

# 🎯 Target Table (Delta)

You have an existing Delta table:

`orders_gold`

```
order_id
customer_id
order_status
order_amount
order_timestamp
last_updated
```

---



# 🧠 Requirements

## Step 1 — Clean & Standardise

* Parse timestamps properly
* Remove records with null order_id
* Remove obviously invalid records (e.g. negative amount)

---


In [0]:
opts = {
    "header":"true",
    "delimiter":",",
    "inferSchema":"true",
    "mode":"PERMISSIVE"
}

df = ( spark.read
      .options(**opts)
      .csv("/Volumes/pyspark/bronze/raw_ingestion/csv/orders_daily_csv/")
)

In [0]:
from pyspark.sql.functions import to_timestamp,col

cleaned_df = (
    df.filter("order_id is not null")
    .filter("order_amount > 0")
    .withColumn("order_timestamp", to_timestamp(col("order_timestamp")))
    .withColumn("last_updated", to_timestamp(col("last_updated")))
)

cleaned_df.display()



## Step 2 — Deduplicate Source

Keep only the latest record per `order_id` using `last_updated`.

---


In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec = Window.partitionBy("order_id").orderBy(col("last_updated").desc())


final_df = (
    cleaned_df.withColumn("rank",F.row_number().over(window_spec))
    .filter("rank == 1")
    .drop("rank")
)

final_df.display()



## Step 3 — Incremental Upsert into Delta

Logic:

* If order_id does NOT exist → INSERT
* If exists AND source.last_updated > target.last_updated → UPDATE
* Otherwise → ignore

---


In [0]:
source_df = final_df

#target_df = spark.read.table("pyspark.silver.t2_orders")


In [0]:
from delta.tables import DeltaTable

target_df = DeltaTable.forName(spark, "pyspark.silver.t2_orders")


(target_df.alias("t")
 .merge(
     source_df.alias("s"),
     "t.order_id = s.order_id"
 )
 .whenMatchedUpdate(
     condition="s.last_updated > t.last_updated",
     set={
         "t.order_status": "s.order_status",
         "t.order_amount": "s.order_amount",
         "t.last_updated": "s.last_updated",
     }
 )
 .whenNotMatchedInsertAll()
 .execute()
)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# combine (make sure schemas align)
combined = target_df.unionByName(source_df, allowMissingColumns=True)

w = Window.partitionBy("order_id").orderBy(F.col("last_updated").desc_nulls_last())

upserted = (combined
            .withColumn("rn", F.row_number().over(w))
            .filter("rn = 1")
            .drop("rn"))

In [0]:
target_df.display()


## Step 4 — Late Arriving Data Handling

Explain:

* What happens if a record from 3 days ago arrives today?
* How do you ensure correctness?

---



## Step 5 — Partition Strategy

Decide:

* How would you partition orders_gold?
* Why?

---



## Step 6 — Performance & Scale

What if:

* 500 million records per day?
* 3 years of history?
* SLA: 30 minutes?

---



# 🚨 What to Test

This task tests:

* MERGE correctness
* Business key logic
* Watermark thinking
* Idempotency
* Partitioning maturity
* Production awareness

---
